# 4 실습: LSTM, GRU

1. 기존 RNN 과 다른 부분에 대해서 배운다.
2. 이전 실습에 이어 다양한 적용법을 배운다.

<br>

## 4.1 필요 패키지 import

In [ ]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

<br>

## 4.2 데이터 전처리

- 아래의 sample data 를 확인해보자. (이전 실습과 동일)

In [ ]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [ ]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 35335.33it/s]


In [ ]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

<br>

## 4.3 LSTM 사용

- LSTM 에선 cell state 가 추가된다.
- cell state 의 shape 은 hidden state 의 shape 과 동일하다.

In [ ]:
embedding_size = 256
hidden_size = 512
num_layers = 1
num_dirs = 1

embedding = nn.Embedding(vocab_size, embedding_size)
lstm = nn.LSTM(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers * num_dirs, B, d_h)

In [ ]:
# d_w: word embedding size
batch_emb = embedding(batch) # (B, L, d_w)
batch_emb.shape

torch.Size([10, 20, 256])

In [ ]:
packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, (h_n, c_n) = lstm(packed_batch, (h_0, c_0))
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)
print(c_n.shape)

PackedSequence(data=tensor([[ 0.0349, -0.0717, -0.0494,  ..., -0.1301,  0.0156, -0.0285],
        [ 0.0039, -0.0848, -0.1199,  ..., -0.0234,  0.1441,  0.0354],
        [ 0.0924,  0.0439, -0.0758,  ...,  0.0022,  0.0554,  0.0257],
        ...,
        [-0.1049,  0.0285,  0.1007,  ..., -0.1961, -0.0256,  0.0783],
        [-0.0058, -0.0997, -0.0244,  ...,  0.1517,  0.2395,  0.0274],
        [-0.0137,  0.1824, -0.2192,  ...,  0.1396,  0.1369,  0.0427]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])


In [ ]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)
print(output_lens)

torch.Size([20, 10, 512])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


<br>

## 4.4 GRU 사용

- GRU 는 cell state 가 없어 RNN 과 동일하게 사용 가능하다.
- GRU 를 이용하여 Language Model task 를 수행해보자.

In [ ]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [ ]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [ ]:
input_id = batch.transpose(0, 1)[0, :] # B
input_id

tensor([85, 58, 87, 22, 70, 19, 93, 94, 80, 62])

In [ ]:
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

<br>

- Teacher forcing 없이 이전에 얻은 결과를 다음 input 으로 이용한다.

In [ ]:
for t in range(max_len):
    input_emb = embedding(input_id).unsqueeze(0) # (1, B, d_w)
    output, hidden = gru(input_emb, hidden) # output: (1, B, d_h), hidden: (1, B, d_h)

    # V: vocab size
    output = output_layer(output) # (1, B, V)
    probs, top_id = torch.max(output, dim=-1) # probs: (1, B), top_id: (1, B)

    print("*" * 50)
    print(f"Time step: {t}")
    print(output.shape)
    print(probs.shape)
    print(top_id.shape)

    input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

<br>

- `max_len` 만큼의 for문을 돌면서 모든 결과물의 모양을 확인했지만 만약 종료 조건(ex. 문장의 끝을 나타내는 end token 등)이 되면 중간에 생성을 그만둘수도 있다.

<br>

## 4.5 양방향 및 여러 layer 사용

- 이번엔 양방향 + 2개 이상의 layer 를 쓸 때 얻을 수 있는 결과에 대해 알아보자.

In [ ]:
num_layers = 2
num_dirs = 2
dropout = 1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

<br>

- Bidirectional 이 되었고 layer 의 개수가 2로 늘었기 때문에 hidden state 의 shape 도 `(4, B, d_h)` 가 된다.

In [ ]:
# d_w: word embedding size
# num_layers: layer 의 개수
# num_dirs: 방향의 개수

batch_emb = embedding(batch) # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[ 0.0406, -0.0119, -0.0071,  ..., -0.2520, -0.1993, -0.1060],
        [ 0.0410,  0.1082, -0.0604,  ..., -0.0563,  0.1071, -0.2233],
        [-0.0133,  0.1561, -0.0856,  ...,  0.2162,  0.0425, -0.0397],
        ...,
        [ 0.0271,  0.0700,  0.1165,  ...,  0.1129, -0.0277,  0.0016],
        [-0.1314, -0.0266,  0.0435,  ..., -0.1107,  0.0446, -0.0148],
        [-0.1458, -0.0240, -0.0314,  ..., -0.0883,  0.0723, -0.0078]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 1024])
torch.Size([4, 10, 512])


In [ ]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


<br>

- 각각의 결과물의 shape는 다음과 같습니다.
  - `outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
  - `h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [ ]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)

tensor([[[[-3.5023e-01, -1.1334e-01, -1.6157e-01,  ...,  4.5304e-01,
            1.8469e-01, -2.1680e-02],
          [-5.7206e-02,  1.3418e-01,  1.2187e-01,  ..., -2.0633e-01,
            1.1726e-01, -1.4778e-01],
          [ 7.1233e-02, -3.1554e-01,  1.7080e-01,  ..., -1.2315e-01,
           -1.9801e-01, -9.2919e-02],
          ...,
          [ 3.1512e-01, -4.1218e-01,  1.7909e-01,  ...,  3.0241e-01,
            4.4074e-01, -1.5008e-01],
          [ 7.7649e-02, -9.8056e-03, -4.0156e-01,  ..., -3.6668e-01,
           -2.1716e-01,  2.9913e-02],
          [ 3.1635e-01,  1.4461e-01,  2.2485e-02,  ..., -2.8990e-01,
            2.8460e-01,  1.2556e-01]],

         [[-1.3612e-01,  4.7373e-01, -2.7130e-01,  ..., -8.1935e-02,
            6.3042e-02, -1.7000e-01],
          [-1.3340e-01, -8.0582e-02, -4.4041e-02,  ...,  9.3998e-02,
            1.4388e-02, -4.7930e-01],
          [-4.2317e-01,  2.9652e-03,  2.6890e-01,  ...,  5.4826e-01,
           -5.1891e-02,  1.4421e-01],
          ...,
     